In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Vehicle_Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(666):
            try:
                if url == '':
                    status = 'ok'
                    
                    break
                    
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_tr = html.xpath('//tr[@class="detail-app-row"]')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                while True:
                    for tr in list_tr:
                        year = int(tr.xpath('./td/text()')[0].strip())
                        make = tr.xpath('./td/text()')[1].strip()
                        model = tr.xpath('./td/text()')[2].strip()
                        engine = tr.xpath('./td/text()')[3].strip()

                        if make not in dict_vehicle:
                            dict_vehicle[make] = {model: {engine: [year]}}
                        else:
                            if model not in dict_vehicle[make]:
                                dict_vehicle[make][model] = {engine: [year]}
                            else:
                                if engine not in dict_vehicle[make][model]:
                                    dict_vehicle[make][model][engine] = [year]
                                else:
                                    dict_vehicle[make][model][engine].append(year)
                            
                    # = = = = = = = = = = = = = = =
                    
                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        break
                    else:
                        resp = requests.get('https://www.dormanproducts.com' + html.xpath('//a[@aria-label="Next"]/@href')[0].strip(),
                                            headers=UA.get_User_Agent_Requests(),
                                            proxies=Proxy.get_Proxy_Requests()).text
                        soup = BeautifulSoup(resp, 'lxml')
                        html = etree.HTML(str(soup))
                        list_tr = html.xpath('//tr[@class="detail-app-row"]')
                        
                # = = = = = = = = = = = = = = =
                        
                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        for engine in dict_vehicle[make][model]: 
                            year_max = max(dict_vehicle[make][model][engine])
                            year_min = min(dict_vehicle[make][model][engine])
                            if year_max == year_min:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Vehicle_Url': url,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Vehicle_Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：991

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=142368&PartType=Catalytic%20Converter  <->  [ok] - 剩余数量：986
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=144969&PartType=Catalytic%20Converter  <->  [ok] - 剩余数量：985
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=141451&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：984
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137555&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：983
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=135603&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：982
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=145576&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：981
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137381&PartType=Catalytic%20Converter%20with%20

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=67895&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：934
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=83883&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：933
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=79180&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：932
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=45452&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：931
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=79181&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：930
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84035&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：929
https://www.dormanproducts.com/itemdetai

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137943&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：882
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=138115&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：881
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=103727&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：880
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=105470&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：879
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=79162&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：878
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=67459&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：877
https://www.dormanproducts.com/itemd

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=129151&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：831
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=132703&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：830
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131785&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：829
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=130235&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：828
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=131431&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：827
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=129795&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：826
https://www.dormanproducts.com/ite

https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102131&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：779
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102136&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：778
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102141&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：777
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102581&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：776
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100181&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：775
https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=86818&PartType=Catalytic%20Converter%20with%20Integrated%20Exhaust%20Manifold  <->  [ok] - 剩余数量：774
https://www.dormanproducts.com/item